In [22]:
import pymysql  
import nbimporter   
import TableCreation 

In [23]:
%run TableCreation.ipynb

Connected successfully!
Table 'patients_details' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'treatment' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'difficulty' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'patient_changes' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'pain_symptoms' created successfully!
Tables: (('difficulty_rati

In [26]:
import cv2
import numpy as np
import easyocr
import json

# Load the image
image_path = "/Users/akhilch/Desktop/example3.png"  # Your image path
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define bounding boxes (field_name: (x, y, width, height))
bounding_boxes = {

    "patient_name": (274, 273, 313, 52),
    "dob": (799, 264, 258, 62),
    "date": (160, 73, 231, 69),  
    "injection": (229, 154, 125, 60),
    "exercise_therapy": (1032, 147, 131, 67),

    "Bending_or_stooping": (324, 400, 153, 59),
    "Putting_on_shoes": (284, 462, 158, 60),
    "Sleeping": (188, 525, 143, 49),



    "Standing_for_an_hour":(332, 582, 141, 50),
    "Going_up_or_down_a_flight_of_stairs":(485, 636, 143, 65),
    "Walking_through_a_store":(370, 699, 150, 62),
    "Driving_for_an_hour":( 305, 755, 156, 60),

    "Preparing_a_meal":(287, 820, 149, 61),
    "Yard_work":(202, 885, 149, 55),
    "Picking_up_itens_off_the_floor": (429, 942, 147, 58),


    

    "since_last_treatment":(93, 1054, 316, 74),
    "since_start_of_treatment":(104, 1172, 420, 74),
    "last_3_days":(114, 1291, 394, 77),


    
    "pain":(146, 1415, 78, 50),
    "numbness":(374, 1411, 78, 53),
    "tingling":(569, 1412, 88, 54),
    "burning":(783, 1415, 79, 54),
    "tightness":(1007, 1408, 61, 59),


    
    "blood_pressure":(289, 1533, 101, 59),
    "hr":(464, 1537, 77, 48),
    "weight":(675, 1533, 110, 56),
    "height":(893, 1532, 104, 57),
    "spo2":(174, 1656, 73, 43),
    "temperature":(439, 1657, 67, 41),
    "blood_glucose":(719, 1655, 82, 47),
    "respirations":(961, 1660, 73, 41),


    
    "program_number":(305, 1604, 68, 45),
    "treatment_number":(631, 1601, 65, 44),
    "placement":(854, 1600, 204, 36),


}

# Initialize OCR reader
reader = easyocr.Reader(['en'])

# Dictionary to store results
output_data = {
    "patient_name": "",
    "dob": "",
    "date": "",
    
    "injection": "",
    "exercise_therapy": "",

    "difficulty_ratings": {
        "bending": "",
        "putting_on_shoes": "",
        "sleeping": "",
        "standing_for_an_hour": "",
        "going_up_or_down_a_flight_of_stairs": "",
        "walking_through_a_store": "",
        "driving_for_an_hour": "",
        "preparing_a_meal": "",
        "yard_work": "",
        "picking_up_items_off_the_floor": ""
    },

    "patient_changes": {
        "since_last_treatment": "",
        "since_start_of_treatment": "",
        "last_3_days": ""
    },

    "pain_symptoms": {
        "pain": "",
        "numbness": "",
        "tingling": "",
        "burning": "",
        "tightness": ""
    },

    "medical_assistant_data": {
        "blood_pressure": "",
        "hr": "",
        "weight": "",
        "height": "",
        "spo2": "",
        "temperature": "",
        "blood_glucose": "",
        "respirations": ""
    }
}

# Extract text for simple fields
for field in ["patient_name", "dob", "date", "since_last_treatment", 
             "since_start_of_treatment", "last_3_days",
              
             "pain","numbness","tingling","burning","tightness","blood_pressure","hr","weight","height",
             "program_number","treatment_number","placement","spo2","temperature","blood_glucose","respirations"]:
    x, y, w, h = bounding_boxes[field]
    roi = gray[y:y+h, x:x+w]
    
    # Run OCR on the extracted region
    result = reader.readtext(roi, detail=0)
    output_data[field] = result[0] if result else ""

# Extract circled numbers
for field in ["injection", "exercise_therapy", "Bending_or_stooping", "Putting_on_shoes", "Sleeping", "Standing_for_an_hour",
             "Going_up_or_down_a_flight_of_stairs", "Walking_through_a_store", "Driving_for_an_hour", "Preparing_a_meal", "Yard_work", 
             "Picking_up_itens_off_the_floor"]:
    x, y, w, h = bounding_boxes[field]
    roi = gray[y:y+h, x:x+w]

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(roi, (9, 9), 2)

    # Detect circles inside the bounding box
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                               param1=50, param2=30, minRadius=10, maxRadius=40)

    if circles is not None:
        circles = np.uint16(np.around(circles))  # Round values
        for (cx, cy, r) in circles[0, :]:
            # Adjust circle coordinates relative to full image
            absolute_x = x + cx
            absolute_y = y + cy

            # Extract the region of interest (ROI) around the detected circle
            circled_roi = gray[y+cy-r:y+cy+r, x+cx-r:x+cx+r]

            # Run OCR on the cropped circle area
            result = reader.readtext(circled_roi, detail=0)
            text = result[0] if result else "?"

            # Store the detected number
            output_data[field] = text

# Convert extracted data to JSON format
json_output = json.dumps(output_data, indent=4)

# Save JSON output to a file
with open("output.json", "w") as json_file:
    json_file.write(json_output)

json_output = json.loads(json_output)
# Print JSON output
print(json_output)



{'patient_name': '8aVi', 'dob': '13-#-95', 'date': '-5-Q4', 'injection': 'YES', 'exercise_therapy': 'N', 'difficulty_ratings': {'bending': '', 'putting_on_shoes': '', 'sleeping': '', 'standing_for_an_hour': '', 'going_up_or_down_a_flight_of_stairs': '', 'walking_through_a_store': '', 'driving_for_an_hour': '', 'preparing_a_meal': '', 'yard_work': '', 'picking_up_items_off_the_floor': ''}, 'patient_changes': {'since_last_treatment': '', 'since_start_of_treatment': '', 'last_3_days': ''}, 'pain_symptoms': {'pain': '', 'numbness': '', 'tingling': '', 'burning': '', 'tightness': ''}, 'medical_assistant_data': {'blood_pressure': '', 'hr': '', 'weight': '', 'height': '', 'spo2': '', 'temperature': '', 'blood_glucose': '', 'respirations': ''}, 'since_last_treatment': 'Good', 'since_start_of_treatment': 'bad', 'last_3_days': 'Wo { st', 'pain': '', 'numbness': '', 'tingling': '3', 'burning': '', 'tightness': '5', 'blood_pressure': '', 'hr': '', 'weight': '', 'height': '', 'program_number': '', 

In [27]:


# Properly formatted JSON output
print(json.dumps(json_output, indent=4))


{
    "patient_name": "8aVi",
    "dob": "13-#-95",
    "date": "-5-Q4",
    "injection": "YES",
    "exercise_therapy": "N",
    "difficulty_ratings": {
        "bending": "",
        "putting_on_shoes": "",
        "sleeping": "",
        "standing_for_an_hour": "",
        "going_up_or_down_a_flight_of_stairs": "",
        "walking_through_a_store": "",
        "driving_for_an_hour": "",
        "preparing_a_meal": "",
        "yard_work": "",
        "picking_up_items_off_the_floor": ""
    },
    "patient_changes": {
        "since_last_treatment": "",
        "since_start_of_treatment": "",
        "last_3_days": ""
    },
    "pain_symptoms": {
        "pain": "",
        "numbness": "",
        "tingling": "",
        "burning": "",
        "tightness": ""
    },
    "medical_assistant_data": {
        "blood_pressure": "",
        "hr": "",
        "weight": "",
        "height": "",
        "spo2": "",
        "temperature": "",
        "blood_glucose": "",
        "respirati

In [28]:
# Assuming json_output is a dictionary containing patient_name and dob
query = """INSERT INTO patient_details (patient_name, dob) 
           VALUES (%s, %s)"""

data = (json_output["patient_name"], json_output["dob"])

try:
    with connection.cursor() as cursor:
        cursor.execute(query, data)  # Safe execution
        patient_id = cursor.lastrowid
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")

✅ Data inserted successfully!


In [29]:
treatment_details = (patient_id,json_output["date"],json_output["injection"],json_output["exercise_therapy"])
treatment_query = """INSERT INTO treatment_details (patient_id, date,injection,exercise_therapy) 
           VALUES (%s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(treatment_query, treatment_details)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(19, '-5-Q4', 'YES', 'N')


In [30]:
print(treatment_details)

(19, '-5-Q4', 'YES', 'N')


In [31]:


difficulty_ratings = (patient_id ,json_output["date"],json_output["Bending_or_stooping"],json_output["Putting_on_shoes"],json_output["Sleeping"])


difficulty_query = """INSERT INTO difficulty_ratings (patient_id, date,bending_or_stooping,putting_on_shoes,sleeping) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(difficulty_query,difficulty_ratings)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(19, '-5-Q4', 'YES', 'N')


In [32]:
print(difficulty_ratings)

(19, '-5-Q4', '?', '(5', '?')


In [33]:


patient_changes = (patient_id ,json_output["date"],json_output["since_last_treatment"],json_output["since_start_of_treatment"],json_output["last_3_days"])


patient_query = """INSERT INTO patient_changes(patient_id, date,since_last_treatment,since_start_of_treatment,last_3_days) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(patient_query,patient_changes)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(19, '-5-Q4', 'YES', 'N')


In [34]:


pain_symptoms= (patient_id ,json_output["date"],json_output["pain"],json_output["numbness"],json_output["tingling"],json_output["burning"],json_output["tightness"])


pain_query = """INSERT INTO pain_symptoms(patient_id, date,pain,numbness,tingling,burning,tightness) 
           VALUES (%s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(pain_query,pain_symptoms)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(19, '-5-Q4', 'YES', 'N')


In [38]:


medical_assistant_data= (patient_id ,json_output["date"],json_output["blood_pressure"],json_output["hr"],json_output["weight"],json_output["height"],json_output["spo2"],json_output["temperature"],
                         json_output["blood_glucose"],json_output["respirations"])


medical_query = """INSERT INTO medical_assistant_data(patient_id, date,blood_pressure,hr,weight,height,spo2,temperature,blood_glucose,respirations) 
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(medical_query,medical_assistant_data)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(19, '-5-Q4', 'YES', 'N')


In [ ]:
},
    "since_last_treatment": "Good",
    "since_start_of_treatment": "bad",
    "last_3_days": "Wo { st",
    "pain": "",
    "numbness": "",
    "tingling": "3",
    "burning": "",
    "tightness": "5",
    "Blood_Pressure": "",
    "HR": "",
    "Weight": "",
    "Height": "",
    "Program_Number": "",
    "Treatment_Number": "",
    "Placement": "",
    "SpO2": "",
    "Temperature": "",
    "Blood_Glucose": "",
    "Respirations": "",
    "Bending_or_stooping": "?",
    "Putting_on_shoes": "(5",
    "Sleeping": "?",
    "Standing_for_an_hour": "(2=",
    "Going_up_or_down_a_flight_of_stairs": "(",
    "Walking_through_a_store": "?",
    "Driving_for_an_hour": "?",
    "Preparing_a_meal": "{4)",
    "Yard_work": "2",
    "Picking_up_itens_off_the_floor": "?"

KeyError: 'patient_id'